In [1]:
import requests as req
import pandas as pd
import os
from tqdm import tqdm

In [2]:
from ipywidgets import HTML

display(
    HTML(
        """
        <style>
            .container {
                width:80vw;
            }
        </style>
        """
    )
)

HTML(value='\n        <style>\n            .container {\n                width:80vw;\n            }\n        <…

In [3]:
# Static variables

URL = "https://api.scryfall.com"
HEADERS = {
    'User-Agent': 'MTGPriceChecker/0.1',
    'Accept': '*/*'
}


# Global variabels for handling some logic

SET_NAME = ""   # Three letter set code of chosen set
DATA = None     # Pandas DataFrame storing card information shuch as owned or not, price, image url, etc.

In [4]:
# Variety of functions for handling card and set logic

def get_card(n):
    """
    Get a spesific card by card number within the chosen set
    n: cardnumber
    """
    return req.get(f"{URL}/cards/{SET_NAME}/{n}").json()


def get_sets():
    """
    Get a list of available Magic the Gathering sets
    """
    return req.get(f"{URL}/sets").json()



def get_set(set_name):
    """
    Get a list of available Magic the Gathering sets
    """
    return req.get(f"{URL}/sets/{SET_NAME}").json()


def save():
    """
    Save the currently open MtG set to .csv format
    """
    
    global SET_NAME, DATA
    if not isinstance(DATA, pd.DataFrame):
        print("No data to save")
        return
    
    if not SET_NAME:
        print("Missing set-name. Can't save current data.")
    
    DATA.to_csv(f"cards_{SET_NAME}.csv")

    
def load_set(set_name):
    """
    Load a MtG set chosen by the user.
    If this is a new set for the user, initialize by using the scryfall api.
    If previously initialized, use the saved DataFrame.
    Always save the previous set before loading a new one.
    Update global variable SET_NAME to the chosen set.
    Update global variable DATA to newly loaded set DataFrame
    set_name: three letter set code
    """
    
    global SET_NAME, DATA
    
    # Save open set in case changes has occured
    if isinstance(DATA, pd.DataFrame):
        save()
    
    # Update set-name
    SET_NAME = set_name
    
    if not os.path.exists(f"cards_{SET_NAME}.csv"):
        res = req.get(f"{URL}/sets/{SET_NAME}")
        NUMBER_OF_CARDS = res.json()['card_count']

        import time as t
        cards = []


        for n in tqdm(range(1, min(NUMBER_OF_CARDS, 20))):
            cards.append(get_card(n))
            t.sleep(.15) # Sleep 15ms () to keep from overloading scryfall API
        
        # Wanted parameters
        params = ['collector_number', 'name', 'promo', 'rarity', 'prices', 'image_uris']
        two_face_params = ['collector_number', 'name', 'promo', 'rarity', 'prices']
        new_params = ['id', 'name', 'promo', 'rarity', 'prices', 'image']


        CARDS = []
        for card in cards:
            if "image_uris" not in card: # Two faced card without default image uris. Use first available face
                new_card = dict([(np, card[p]) for np, p in zip(new_params, two_face_params)])
                new_card['image'] = card['card_faces'][0]['image_uris']
            else:
                new_card = dict([(np, card[p]) for np, p in zip(new_params, params)])
                
            new_card['id'] = int(new_card['id'])
            new_card['image'] = new_card['image']['normal']
            new_card['standard_price'] = new_card['prices']['usd']
            new_card['foil_price'] = new_card['prices']['usd_foil']
            del new_card['prices']

            new_card['owned'] = False

            CARDS.append(new_card)

        DATA = pd.DataFrame.from_records(CARDS)
        DATA.to_csv(f"cards_{SET_NAME}.csv", index=False)

        del cards
        del CARDS
        del res
        del NUMBER_OF_CARDS

    DATA = pd.read_csv(f"cards_{set_name}.csv", index_col="id")
    
    return DATA


def update_prices():
    """
    Prices may change. Update to get current average market prices
    """
    pass

In [5]:
# Reusable HTML components

def set_list(sets):
    return HTML(f"""
        <div class='set-list'>
            {[
                set_card(SET)
                for SET in sets
            ]}
        </div>
    """)

def set_card(set_object):
    return HTML(f"""
        <div>
            <h2>{set_object['name']} ({set_object['code']})</h2>
            {set_object['card_count']} cards, released {set_object['released_at']}
        </div>
    """)

In [ ]:
# MTG folio ipywidgets components
# Paste into a Jupyter notebook cell and run.
# If ipywidgets isn't installed: pip install ipywidgets

from IPython.display import display
import html
from textwrap import dedent

try:
    import ipywidgets as widgets
except Exception as e:
    raise RuntimeError("ipywidgets is required. Install with `pip install ipywidgets`.") from e

# ------------------------------
# Stylesheet (large, separate)
# ------------------------------
STYLES = dedent(r"""
:root{
  --folio-bg: #0b0b0b;
  --card-border: rgba(255,255,255,0.06);
  --card-radius: 6px;
  --card-shadow: 0 6px 18px rgba(0,0,0,0.5);
  --gap: 10px;
  --page-padding: 18px;
  --card-w-3x3: 210px;
  --card-w-4x4: 150px;
  --card-aspect: 1.395;
  --detail-img-max-w: 280px;
  --accent: linear-gradient(135deg, #f6b93b, #f39c12);
  --btn-bg: rgba(255,255,255,0.06);
  --btn-radius: 6px;
}
.folio-frontpage{ display:block; position: relative; width: fit-content; margin: 12px auto; overflow: hidden; border-radius: 8px; box-shadow: var(--card-shadow); background: #111; }
.folio-frontpage[data-grid="3x3"]{ width: calc(var(--card-w-3x3) * 3 + var(--gap) * 4 + var(--page-padding) * 2); }
.folio-frontpage[data-grid="4x4"]{ width: calc(var(--card-w-4x4) * 4 + var(--gap) * 5 + var(--page-padding) * 2); }
.folio-frontpage-image{ display:block; width:100%; height: auto; object-fit: cover; vertical-align: middle; border-radius: 8px; }
.folio-page{ display:block; margin: 12px auto; background: linear-gradient(180deg, rgba(255,255,255,0.02), rgba(0,0,0,0.02)); padding: var(--page-padding); border-radius: 8px; box-shadow: var(--card-shadow); width: fit-content; }
.folio-grid{ display: grid; gap: var(--gap); background: transparent; }
.folio-slot{ display:flex; align-items:center; justify-content:center; background: transparent; padding: 2px; border-radius: 4px; box-sizing: border-box; }
.single-card-container{ width:100%; height:100%; display:flex; align-items:center; justify-content:center; background: rgba(255,255,255,0.02); border: 1px solid var(--card-border); border-radius: 6px; overflow: hidden; }
.single-card-image{ display:block; max-width:100%; max-height:100%; object-fit: contain; width: 100%; height: auto; image-rendering: auto; }
.folio-page[data-grid="3x3"] .folio-slot{ width: var(--card-w-3x3); height: calc(var(--card-w-3x3) * var(--card-aspect)); }
.folio-page[data-grid="4x4"] .folio-slot{ width: var(--card-w-4x4); height: calc(var(--card-w-4x4) * var(--card-aspect)); }
.card-detail{ display:flex; flex-direction: column; gap: 12px; align-items: center; justify-content: flex-start; padding: 12px; border-radius: 8px; background: linear-gradient(180deg, rgba(0,0,0,0.45), rgba(0,0,0,0.38)); color: #fff; width: fit-content; box-shadow: var(--card-shadow); border: 1px solid rgba(255,255,255,0.04); font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial; }
.card-detail-imagewrap{ width: 100%; display:flex; align-items:center; justify-content:center; }
.card-detail-image{ width: 100%; max-width: var(--detail-img-max-w); height: auto; border-radius: 6px; border: 1px solid rgba(255,255,255,0.06); box-shadow: 0 8px 22px rgba(0,0,0,0.55); }
.card-detail-title{ margin: 0; padding: 0; font-size: 1.05rem; letter-spacing: 0.2px; text-align: center; color: #f3f3f3; }
.card-detail-price{ font-weight: 600; color: #ffd480; font-size: 0.95rem; }
.btn{ padding: 8px 10px; background: var(--btn-bg); border-radius: var(--btn-radius); border: 1px solid rgba(255,255,255,0.04); cursor: pointer; font-weight: 600; color: #fff; user-select:none; }
.btn:active{ transform: translateY(1px); }
.controls-row{ display:flex; gap:8px; align-items:center; justify-content:center; }
.counter{ display:flex; align-items:center; gap:8px; padding:6px 8px; border-radius: 10px; background: linear-gradient(180deg, rgba(255,255,255,0.02), rgba(0,0,0,0.02)); border: 1px solid rgba(255,255,255,0.03); }
.counter .cnt-value{ min-width: 36px; text-align:center; font-weight:700; font-size:1rem; }
.small-muted{ font-size:0.82rem; color: rgba(255,255,255,0.65); }
""")

# ------------------------------
# Widgets / Factories
# ------------------------------
def stylesheet_widget():
    """Return an ipywidgets.HTML that injects the main stylesheet. Display this once per notebook."""
    return widgets.HTML(f"<style>{html.escape(STYLES)}</style>")

def create_front_page(set_image_url, grid='3x3'):
    """Return ipywidgets.HTML: front page full-bleed image. grid='3x3' or '4x4'"""
    if grid not in ('3x3','4x4'):
        raise ValueError("grid must be '3x3' or '4x4'")
    esc = html.escape(set_image_url or '')
    html_str = dedent(f"""
    <div class="folio-frontpage" data-grid="{grid}">
      <img class="folio-frontpage-image" src="{esc}" alt="Set image">
    </div>
    """)
    return widgets.HTML(value=html_str)

def create_page(grid='3x3'):
    """Return ipywidgets.HTML: an empty page with slots for cards."""
    if grid not in ('3x3','4x4'):
        raise ValueError("grid must be '3x3' or '4x4'")
    cols = 3 if grid == '3x3' else 4
    rows = cols
    slots_html = ""
    for i in range(rows * cols):
        slots_html += f'<div class="folio-slot" data-index="{i}"><div class="folio-card" data-slot="{i}"></div></div>\n'
    grid_html = f'<div class="folio-page" data-grid="{grid}"><div class="folio-grid" style="grid-template-columns: repeat({cols}, 1fr); grid-template-rows: repeat({rows}, 1fr);">{slots_html}</div></div>'
    return widgets.HTML(value=grid_html)

def create_card(image_url, opacity=1.0):
    """Return ipywidgets.HTML: single-card container showing only the card image.
       opacity: 0.0 - 1.0"""
    if not (0 <= opacity <= 1):
        raise ValueError("opacity must be between 0 and 1")
    esc = html.escape(image_url or '')
    html_str = dedent(f"""
    <div class="single-card-container" style="opacity: {opacity};">
      <img class="single-card-image" src="{esc}" alt="Card image">
    </div>
    """)
    return widgets.HTML(value=html_str)

def create_card_detail_view(card):
    """Return ipywidgets.HTML: detail view with image, title, price, owned/foil toggles and two counters.
       card: dict with keys:
         - name (str)
         - imageUrl (str)
         - price (str or number) optional
         - owned (bool) optional
         - foil (bool) optional
         - counts (dict) optional, e.g. {'normal':1,'foil':0}
    """
    name = html.escape(str(card.get('name','Unknown')))
    imageUrl = html.escape(card.get('imageUrl',''))
    price = card.get('price', '—')
    owned = bool(card.get('owned', False))
    foil = bool(card.get('foil', False))
    counts = card.get('counts', {'normal':0,'foil':0})
    normal_count = int(counts.get('normal', 0))
    foil_count = int(counts.get('foil', 0))

    template = """
    <div class="card-detail" data-card-name="{name}">
      <div class="card-detail-imagewrap">
        <img class="card-detail-image" src="{imageUrl}" alt="{name}">
      </div>
      <h3 class="card-detail-title">{name}</h3>
      <div class="card-detail-price">{price_display}</div>

      <div class="controls-row" style="gap:10px">
        <button class="btn btn-toggle-owned" data-owned="{owned}">{owned_label}</button>
        <button class="btn btn-toggle-foil" data-foil="{foil}">{foil_label}</button>
      </div>

      <div class="controls-row" style="margin-top:8px">
        <div class="counter" data-type="normal">
          <button class="btn btn-decr" type="button">-</button>
          <div class="cnt-value">{normal_count}</div>
          <button class="btn btn-incr" type="button">+</button>
          <div class="small-muted" style="margin-left:6px">normal owned</div>
        </div>

        <div class="counter" data-type="foil">
          <button class="btn btn-decr" type="button">-</button>
          <div class="cnt-value">{foil_count}</div>
          <button class="btn btn-incr" type="button">+</button>
          <div class="small-muted" style="margin-left:6px">foil owned</div>
        </div>
      </div>

      <script>
      (function(){{
        const root = document.currentScript ? document.currentScript.parentElement : null;
        if(!root) return;
        const ownedBtn = root.querySelector('.btn-toggle-owned');
        if(ownedBtn){{
          ownedBtn.addEventListener('click', function(){{
            const v = ownedBtn.getAttribute('data-owned') === 'true';
            ownedBtn.setAttribute('data-owned', (!v).toString());
            ownedBtn.textContent = (!v) ? 'Owned' : 'Needed';
          }});
        }}
        const foilBtn = root.querySelector('.btn-toggle-foil');
        if(foilBtn){{
          foilBtn.addEventListener('click', function(){{
            const v = foilBtn.getAttribute('data-foil') === 'true';
            foilBtn.setAttribute('data-foil', (!v).toString());
            foilBtn.textContent = (!v) ? 'Foil' : 'Normal';
          }});
        }}
        const counters = root.querySelectorAll('.counter');
        counters.forEach(function(cnt){{
          const incr = cnt.querySelector('.btn-incr');
          const decr = cnt.querySelector('.btn-decr');
          const valEl = cnt.querySelector('.cnt-value');
          let value = parseInt(valEl.textContent || '0', 10) || 0;
          incr && incr.addEventListener('click', function(){{ value += 1; valEl.textContent = value; }});
          decr && decr.addEventListener('click', function(){{ value = Math.max(0, value - 1); valEl.textContent = value; }});
        }});
      }})();
      </script>
    </div>
    """

    price_display = ("$" + str(price)) if price != '—' else '—'
    html_final = template.format(
        name=name, imageUrl=imageUrl, price_display=price_display,
        owned=str(owned).lower(), owned_label=('Owned' if owned else 'Needed'),
        foil=str(foil).lower(), foil_label=('Foil' if foil else 'Normal'),
        normal_count=normal_count, foil_count=foil_count
    )
    return widgets.HTML(value=html_final)

# Exported convenience display for demonstration (optional)
if __name__ == "__main__":
    display(widgets.HTML("<b>MTG folio stylesheet (display once)</b>"))
    display(stylesheet_widget())
    display(widgets.HTML("<b>Example front page (3x3)</b>"))
    display(create_front_page("https://images.unsplash.com/photo-1549880338-65ddcdfd017b?w=1200", grid='3x3'))
    display(widgets.HTML("<b>Example page (4x4)</b>"))
    display(create_page('4x4'))
    display(widgets.HTML("<b>Example single card</b>"))
    display(create_card("https://cards.scryfall.io/normal/front/2/5/25a06f82-ebdb-4dd6-bfe8-958018ce557c.jpg", opacity=0.9))
    display(widgets.HTML("<b>Example card detail view</b>"))
    sample_card = {'name': 'Sample Card', 'imageUrl': 'https://archive.org/download/mtg-sample-card/mtg-sample-card.png', 'price': '3.50', 'owned': False, 'foil': False, 'counts': {'normal':1,'foil':0}}
    display(create_card_detail_view(sample_card))

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import html
from textwrap import dedent

def mtg_set_paginated_carousel(set_list, visible_count=5, page=0):
    """
    Create a paginated horizontal MTG set carousel for Jupyter notebooks.

    Args:
        set_list (list of dict): Each dict has {'name': str, 'icon_svg_uri': str}
        visible_count (int): How many sets to show per page
        page (int): starting page (0-based)
    Returns:
        ipywidgets.HTML
    """
    # Escape names/urls
    for s in set_list:
        s['name'] = html.escape(s['name'])
        s['icon_svg_uri'] = html.escape(s['icon_svg_uri'])

    # Build HTML for each card
    set_items = ""
    for s in set_list:
        set_items += f"""
        <div class="carousel-item">
            <h3 class="set-title">{s['name']}</h3>
            <div class="set-image-wrap">
                <img src="{s['icon_svg_uri']}" alt="{s['name']}">
            </div>
        </div>
        """

    html_str = f"""
    <style>
    .carousel-wrapper {{
        position: relative;
        width: 100%;
        overflow: hidden;
        padding: 20px 40px;
        box-sizing: border-box;
    }}
    .carousel-container {{
        display: flex;
        gap: 20px;
        transition: transform 0.4s ease;
    }}
    .carousel-item {{
        flex: 0 0 auto;
        width: 50px;
        background: #111;
        border-radius: 8px;
        padding: 12px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.5);
        color: #fff;
        text-align: center;
    }}
    .carousel-item h3.set-title {{
        margin: 0 0 8px 0;
        font-size: 1rem;
        font-weight: 600;
        line-height: 1.2;
    }}
    .set-image-wrap {{
        display: flex;
        align-items: center;
        justify-content: center;
        height: 120px;
    }}
    .set-image-wrap img {{
        max-width: 100%;
        max-height: 100%;
        border-radius: 4px;
    }}
    /* Half fade effect */
    .carousel-wrapper::before, .carousel-wrapper::after {{
        content: "";
        position: absolute;
        top: 0; bottom: 0;
        width: 40px;
        pointer-events: none;
        z-index: 2;
    }}
    .carousel-wrapper::before {{
        left: 0;
        background: linear-gradient(to right, rgba(0,0,0,0.5), transparent);
    }}
    .carousel-wrapper::after {{
        right: 0;
        background: linear-gradient(to left, rgba(0,0,0,0.5), transparent);
    }}
    .carousel-arrow {{
        position: absolute;
        top: 50%;
        transform: translateY(-50%);
        background: rgba(255,255,255,0.1);
        border: none;
        border-radius: 4px;
        color: #fff;
        font-weight: bold;
        padding: 6px 10px;
        cursor: pointer;
        z-index: 10;
    }}
    .carousel-arrow.left {{ left: 10px; }}
    .carousel-arrow.right {{ right: 10px; }}
    </style>

    <div class="carousel-wrapper">
        <button class="carousel-arrow left" id="carousel-prev">←</button>
        <div class="carousel-container" id="carousel-container">
            {set_items}
        </div>
        <button class="carousel-arrow right" id="carousel-next">→</button>
    </div>

    <script>
    (function(){{
        const container = document.getElementById('carousel-container');
        const btnNext = document.getElementById('carousel-next');
        const btnPrev = document.getElementById('carousel-prev');
        const visibleCount = {visible_count};
        const totalItems = {len(set_list)};
        const itemWidth = container.querySelector('.carousel-item').offsetWidth + 20; // including gap
        let page = {page};
        const maxPage = Math.ceil(totalItems / visibleCount) - 1;

        function updateCarousel(){{
            if(page < 0) page = 0;
            if(page > maxPage) page = maxPage;
            container.style.transform = 'translateX(-' + (page * visibleCount * itemWidth) + 'px)';
            
            // Show/hide arrows
            btnPrev.style.display = page === 0 ? 'none' : 'block';
            btnNext.style.display = page === maxPage ? 'none' : 'block';
        }}

        btnNext.addEventListener('click', function(){{
            page += 1;
            updateCarousel();
            // optionally send page back to Python
        }});
        btnPrev.addEventListener('click', function(){{
            page -= 1;
            updateCarousel();
        }});

        // initial render
        updateCarousel();
    }})();
    </script>
    """
    return widgets.HTML(value=html_str)

In [ ]:
sets = get_sets()['data']

display(mtg_set_paginated_carousel(sets, visible_count=5))